In [1]:
# really importans stuff: this gets us time at utc 15 minutes ago
import pendulum
pendulum.now().set(second=0, microsecond=0).in_timezone("UTC").subtract(minutes=15)

DateTime(2026, 1, 4, 5, 3, 0, tzinfo=Timezone('UTC'))

In [6]:
pendulum.today().in_timezone("UTC").date().isoformat()

'2026-01-03'

In [12]:
pendulum.now().set(second=0, microsecond=0).in_timezone("UTC").subtract(minutes=15).isoformat()

'2026-01-04T07:51:00+00:00'

In [9]:
import string
alphabet=list(string.ascii_lowercase)
letter = "x"
if alphabet.index(letter)+10 > len(alphabet):
    for symbol in alphabet[alphabet.index(letter)+1:]:
        print(f"process {symbol}")
else:
    for symbol in alphabet[alphabet.index(letter)+1:alphabet.index(letter)+11]:
        print(f"process {symbol}")


process y
process z


In [10]:
import requests

url = "https://data.alpaca.markets/v2/stocks/bars?symbols=aapl&timeframe=1Min&start=2026-01-03&end=2026-01-03&limit=1000&adjustment=raw&feed=sip&page_token=&sort=asc"

headers = {
    "accept": "application/json",
    "APCA-API-KEY-ID": "PKCTYL4MO5SA2QEIZL2TDEJRTA",
    "APCA-API-SECRET-KEY": "DKL2eLVYFcxKzMJtDYbuG3zppWjpwHzsTS1DtHVwc9Cz"
}

response = requests.get(url, headers=headers)

print(response.text)

{"bars":{"AAPL":[{"c":270.9795,"h":270.9795,"l":270.9605,"n":12,"o":270.9605,"t":"2026-01-03T00:00:00Z","v":451,"vw":270.968761},{"c":270.9632,"h":270.9632,"l":270.9632,"n":8,"o":270.9632,"t":"2026-01-03T00:02:00Z","v":207,"vw":270.9632},{"c":270.9609,"h":270.9609,"l":270.96,"n":6,"o":270.96,"t":"2026-01-03T00:11:00Z","v":460,"vw":270.960273},{"c":270.98,"h":270.98,"l":270.97,"n":8,"o":270.97,"t":"2026-01-03T00:14:00Z","v":518,"vw":270.976},{"c":270.98,"h":270.98,"l":270.98,"n":17,"o":270.98,"t":"2026-01-03T00:16:00Z","v":973,"vw":270.98},{"c":270.98,"h":270.98,"l":270.96,"n":10,"o":270.9632,"t":"2026-01-03T00:17:00Z","v":323,"vw":270.967733},{"c":270.96,"h":270.96,"l":270.96,"n":3,"o":270.96,"t":"2026-01-03T00:19:00Z","v":274,"vw":270.96},{"c":270.96,"h":270.96,"l":270.96,"n":16,"o":270.96,"t":"2026-01-03T00:20:00Z","v":295,"vw":270.96},{"c":270.9599,"h":270.96,"l":270.9599,"n":7,"o":270.96,"t":"2026-01-03T00:24:00Z","v":236,"vw":270.95995},{"c":270.9,"h":270.9,"l":270.9,"n":9,"o":270

In [11]:
type(parsed['bars'])

dict

In [15]:
# Get tickers
tickers = get_tickers()
latest_bars = get_latest_bars(tickers, headers_alpaca)
type(latest_bars)

dict

In [35]:
# no taskflow
import pendulum
import tasks_alpaca

from airflow.providers.standard.operators.python import PythonOperator
from airflow.sdk import DAG
from minio import Minio
from pathlib import Path

# minio conn
login_minio, password_minio = tasks_alpaca.get_minio_credentials()
host = "localhost"
client = Minio(
    endpoint=f"{host}:9000",
    access_key=login_minio,
    secret_key=password_minio,
    secure=False
    )


with DAG(
    dag_id="get_latest_bars",
    catchup=False,
    tags="stonks",
    start_date=pendulum.now().set(second=0, microsecond=0).in_timezone("UTC").subtract(minutes=15),
    schedule=pendulum.duration(minutes=1)
) as dag:
    
    tickers = PythonOperator(
        task_id="get_ticker_list",
        python_callable=tasks_alpaca.get_tickers,
        op_kwargs={"path_to_json": Path().cwd() / "data" / "tickers.json"},
        dag=dag)
    bars = PythonOperator(
        task_id="get_latest_bars",
        python_callable=tasks_alpaca.get_latest_bars,
        op_kwargs={"tickers_to_search": tickers},
        dag=dag)
    save_data = PythonOperator(
        task_id="save_data_to_bronze_bucket",
        python_callable=tasks_alpaca.upload_to_bucket,
        op_kwargs={
            "source_file":bars,
            "bucket_name":"learning.bronze",
            "client": client,
            "destination_file":dag.start_date
        })
    

2025-12-16 04:10:00+00:00


In [44]:
# realtime taskflow
import io
import json
import os
import pendulum 
import requests

from airflow.sdk import dag, task
from dotenv import load_dotenv
from minio import Minio
from minio.error import S3Error
from pathlib import Path
from tasks_alpaca import get_minio_credentials, create_bucket
from typing import Union, Tuple, Optional
from urllib.parse import urlencode

credentials = {
        "key": "PKCTYL4MO5SA2QEIZL2TDEJRTA",
        "secret_key": "DKL2eLVYFcxKzMJtDYbuG3zppWjpwHzsTS1DtHVwc9Cz"
    }
headers_alpaca = {
    "accept": "application/json",
    "APCA-API-KEY-ID": credentials["key"],
    "APCA-API-SECRET-KEY": credentials["secret_key"]
}

# minio conn
login_minio, password_minio = get_minio_credentials()
host = "localhost"
client = Minio(
    endpoint=f"{host}:9001",
    access_key=login_minio,
    secret_key=password_minio,
    secure=False
    )

def get_realtime_tickers():
    @task
    def get_tickers(path_to_json: Optional[Path] = None) -> list:
        if path_to_json is None:
            return ['aapl', 'nvda']
        with open(path_to_json, "r") as tickers_file:
            tickers = json.load(tickers_file)
        return tickers
    @task
    def get_latest_bars(tickers_to_search):
        print(tickers_to_search)
        params = {
            'symbols': ",".join(tickers_to_search),
            }
        url = f"https://data.alpaca.markets/v2/stocks/bars/latest?{urlencode(params)}"
        response = requests.get(url, headers=headers_alpaca)
        return response.text
    @task    
    def upload_to_bucket(source_file: Union[str, Path, bytes], bucket_name: str, client: Minio, destination_file: str=None) -> None:
        # Make the bucket if it doesn't exist.
        create_bucket(bucket_name, client)
        # Encode str to bytes
        file_encoded = source_file.encode("utf-8")
        file_to_upload = io.BytesIO(file_encoded)
        # Upload file
        client.put_object(
            data=file_to_upload,
            bucket_name=bucket_name,
            object_name=destination_file,
            length=len(file_encoded)
        )
        print("Successfully uploaded object", destination_file, "to bucket", bucket_name)

    tickers = get_tickers()
    bars = get_latest_bars(tickers)
    upload_to_bucket(source_file=bars, bucket_name="airflow.learn", client=client, destination_file="{{dag_run.start_date}}")
    

In [45]:
get_realtime_tickers()

['aapl', 'nvda']
Bucket airflow.learn already exists
Successfully uploaded object test_upload.txt to bucket airflow.learn


In [46]:
Path("airflow_learning/data/tickers.json")

PosixPath('airflow_learning/data/tickers.json')

In [48]:
pwd

'/home/heinz/python-workspace/stonks/airflow_learning/dags'

In [50]:
with open (Path("/home/heinz/python-workspace/stonks/data/latest"), "rb") as file:
    bars = file.read().decode()
bars

'asss'

In [ ]:
msft 15:00

In [ ]:
# realtime taskflow
import io
import json
import os
import pendulum 
import requests

from airflow.sdk import dag, task
from dotenv import load_dotenv
from minio import Minio
from minio.error import S3Error
from pathlib import Path
from tasks_alpaca import get_minio_credentials, create_bucket
from typing import Union, Tuple, Optional
from urllib.parse import urlencode


credentials = {
        "key": "PKCTYL4MO5SA2QEIZL2TDEJRTA",
        "secret_key": "DKL2eLVYFcxKzMJtDYbuG3zppWjpwHzsTS1DtHVwc9Cz"
    }
headers_alpaca = {
    "accept": "application/json",
    "APCA-API-KEY-ID": credentials["key"],
    "APCA-API-SECRET-KEY": credentials["secret_key"]
}

# minio conn
login_minio, password_minio = get_minio_credentials()
host = "localhost"
client = Minio(
    endpoint=f"{host}:9001",
    access_key=login_minio,
    secret_key=password_minio,
    secure=False
    )

def get_latest_bars(tickers_to_search):
    print(tickers_to_search)
    params = {
        'symbols': ",".join(tickers_to_search),
        }
    url = f"https://data.alpaca.markets/v2/stocks/bars/latest?{urlencode(params)}"
    response = requests.get(url, headers=headers_alpaca)
    return response.text

первый запуск: 
1) собираем историю за сегодня и берем последний тик
2) собираем историю за последние 80 дней с группировкой по 3 часа
3) повторяем 2 на нужном периоде:
    - смотрим самую раннюю дату на файлах и берем её как энд_дэйт, от нее последние 80 дней собираем
4) когда собрали за нужный период, увеличиваем зернистость: по 1 часу, после - по минуте
5) Чтобы не искать каждый раз раннюю дату, можно сделать какой-нибудь флаг, который показывает, где даг остановился

In [ ]:
def find_earliest_bars(path_to_bars: Union[str, Path]) -> pendulum.Date:
    end_date = pendulum.today().date()
    bars_items = list(Path(path_to_bars).iterdir())
    if len(bars_items) > 0:
        #find earliest date
        for item in bars_items:
            date_str = item.name.split("-")[0].split("_")[1]
            date_parsed = pendulum.parse(date_str, tz="UTC").date()
            if date_parsed < end_date:
                end_date = date_parsed
    return end_date
    
def get_historical_bars(
    tickers_to_search: list[str],
    time_frame: str = "3H",
    date_start: str = None,
    date_end: str = None,
    limit: int = 10000,
    page_token: str = "") -> Tuple[str, bool]:
    """ 
        Saves json in file or prints errors made in arguments.
        Args:
            tickers_to_search:
                non-empty list of stock tickers.
            time_frame:
                [1-59]Min or [1-59]T, e.g. 5Min or 5T creates 5-minute aggregations.
                [1-23]Hour or [1-23]H, e.g. 12Hour or 12H creates 12-hour aggregations.
                1Day or 1D creates 1-day aggregations.
                1Week or 1W creates 1-week aggregations.
                [1,2,3,4,6,12]Month or [1,2,3,4,6,12]M, e.g. 3Month or 3M creates 3-month aggregations.
            date_start and date-end:
                string in YYYY-MM-DD or rfc-3339 format. date_start should be an earlier date than date_end.
            limit: 
                number between 1 and 10000.
            page_token:
                a string you get from 'next_page_token' field of response.
                If this is the first page of your call, then leave it as empty string.
                If there is no next page for your call, 'next_page_token' field will return None value. This behavior is expected.
        Returns:
            Tuple(bars, is_there_next_page) 
                """
        
    # # Validate arguments
    # is_valid, error_messages = validate_arguments(tickers_to_search, date_start, date_end, time_frame, limit)
    # if not is_valid:
    #     for error in error_messages:
    #         print(error)
    #     return False
    
    params = {
    'symbols': ",".join(tickers_to_search),
    'timeframe': time_frame, 
    'start': date_start,
    'end': date_end,
    'limit': limit,
    'adjustment': 'raw',
    'feed': 'sip', 
    'page_token': "",
    'sort': "asc"
    }
    while params['page_token'] is not None: # while there is next page
        url = f"https://data.alpaca.markets/v2/stocks/bars?{urlencode(params)}"
        response = requests.get(url, headers=headers_alpaca)
        parsed = json.loads(response.text)
        params['page_token'] = parsed['next_page_token']
    return True

In [3]:
from pathlib import Path
items = list(Path().cwd().iterdir())
items[0].name

'my_test_dag.py'

In [7]:
len(Path().cwd().iterdir())

TypeError: object of type 'generator' has no len()

In [6]:
import pendulum
date="2025-12-10T10:14:00Z"
pendulum.parse(date,tz="UTC")

DateTime(2025, 12, 10, 10, 14, 0, tzinfo=Timezone('UTC'))

In [ ]:
    tickers_to_search: list[str],
    time_frame: str = "3H",
    date_start: str = None,
    date_end: str = None,
    limit: int = 10000,
    page_token: str = "") -> Tuple[str, bool]:
date_end = find_earliest_bars(#path)
get_historical_bars(tickers, time_frame="3H", date_start=date_end.subtract(days=80), date_end=date_end)